# BPE Tutorial

> This NB contains the code for training a BPE

In [1]:
import pandas as pd

from icodegen.data.core import *
from pathlib import Path
from sklearn.model_selection import train_test_split

In [4]:
path = Path('../dvc-icodegen')
df = pd.read_csv(path/'searchnet/[codesearchnet-java-1597073966.81902].csv', sep = '~')#, index_col = 0)
df.head()

,repo,path,url,code,code_tokens,docstring,docstring_tokens,language,partition,bpe32k,code_len,bpe32_len
0,apache/spark,sql/hive-thriftserver/src/main/java/org/apache...,https://github.com/apache/spark/blob/25ee0474f...,<@>Override<n> <public> OperationHandle getSc...,"['@', 'Override', 'public', 'OperationHandle',...",/* (non-Javadoc)\n@see org.apache.hive.service...,"['/', '*', '(', 'non', '-', 'Javadoc', ')']",java,train,"['▁', '@', 'Override', '▁public', '▁Operation'...",31.0,43.0
1,BlueBrain/bluima,modules/bluima_typesystem/src/main/java/de/jul...,https://github.com/BlueBrain/bluima/blob/793ea...,<public> <void> setTextObjects<(>FSArray v<)> ...,"['public', 'void', 'setTextObjects', '(', 'FSA...",setter for textObjects - sets the text objects...,"['setter', 'for', 'textObjects', '-', 'sets', ...",java,train,"['▁public', '▁void', '▁setText', 'Objects', '(...",64.0,92.0
2,aws/aws-sdk-java,aws-java-sdk-athena/src/main/java/com/amazonaw...,https://github.com/aws/aws-sdk-java/blob/aa385...,<public> <void> marshall<(>WorkGroup workGroup...,"['public', 'void', 'marshall', '(', 'WorkGroup...",Marshall the given parameter object.,"['Marshall', 'the', 'given', 'parameter', 'obj...",java,train,"['▁public', '▁void', '▁marshall', '(', 'Work',...",117.0,140.0
3,softindex/datakernel,core-bytebuf/src/main/java/io/datakernel/byteb...,https://github.com/softindex/datakernel/blob/0...,<public> <void> put<(><@>NotNull <byte><[><]> ...,"['public', 'void', 'put', '(', '@', 'NotNull',...",Puts given byte array to the {@link ByteBuf} f...,"['Puts', 'given', 'byte', 'array', 'to', 'the'...",java,train,"['▁public', '▁void', '▁put', '(', '@', 'NotNul...",64.0,72.0
4,lunisolar/magma,magma-func-builder/src/main/java/eu/lunisolar/...,https://github.com/lunisolar/magma/blob/83809c...,<@>Non<null><n>\t<public> <static> <lesser>T<g...,"['@', 'Nonnull', 'public', 'static', '<', 'T',...",One of ways of creating builder. This is possi...,"['One', 'of', 'ways', 'of', 'creating', 'build...",java,train,"['▁', '@', 'Nonnull', '▁public', '▁static', '▁...",46.0,52.0


In [5]:
len(df)

766407

In [7]:
len(df.drop_duplicates())

650786

In [5]:
# df_trn = df[df.partition == 'train'].copy()
# df_val = df[df.partition == 'valid'].copy()
# df_tst = df[df.partition == 'test'].copy()
# print(len(df_trn))
# df_trn.head()

384868


,path,url,code,code_tokens,docstring,docstring_tokens,language,partition,bpe32k,code_len,bpe32_len
repo,,,,,,,,,,,
apache/spark,sql/hive-thriftserver/src/main/java/org/apache...,https://github.com/apache/spark/blob/25ee0474f...,<@>Override<n> <public> OperationHandle getSc...,"['@', 'Override', 'public', 'OperationHandle',...",/* (non-Javadoc)\n@see org.apache.hive.service...,"['/', '*', '(', 'non', '-', 'Javadoc', ')']",java,train,"['▁', '@', 'Override', '▁public', '▁Operation'...",31.0,43.0
BlueBrain/bluima,modules/bluima_typesystem/src/main/java/de/jul...,https://github.com/BlueBrain/bluima/blob/793ea...,<public> <void> setTextObjects<(>FSArray v<)> ...,"['public', 'void', 'setTextObjects', '(', 'FSA...",setter for textObjects - sets the text objects...,"['setter', 'for', 'textObjects', '-', 'sets', ...",java,train,"['▁public', '▁void', '▁setText', 'Objects', '(...",64.0,92.0
aws/aws-sdk-java,aws-java-sdk-athena/src/main/java/com/amazonaw...,https://github.com/aws/aws-sdk-java/blob/aa385...,<public> <void> marshall<(>WorkGroup workGroup...,"['public', 'void', 'marshall', '(', 'WorkGroup...",Marshall the given parameter object.,"['Marshall', 'the', 'given', 'parameter', 'obj...",java,train,"['▁public', '▁void', '▁marshall', '(', 'Work',...",117.0,140.0
softindex/datakernel,core-bytebuf/src/main/java/io/datakernel/byteb...,https://github.com/softindex/datakernel/blob/0...,<public> <void> put<(><@>NotNull <byte><[><]> ...,"['public', 'void', 'put', '(', '@', 'NotNull',...",Puts given byte array to the {@link ByteBuf} f...,"['Puts', 'given', 'byte', 'array', 'to', 'the'...",java,train,"['▁public', '▁void', '▁put', '(', '@', 'NotNul...",64.0,72.0
lunisolar/magma,magma-func-builder/src/main/java/eu/lunisolar/...,https://github.com/lunisolar/magma/blob/83809c...,<@>Non<null><n>\t<public> <static> <lesser>T<g...,"['@', 'Nonnull', 'public', 'static', '<', 'T',...",One of ways of creating builder. This is possi...,"['One', 'of', 'ways', 'of', 'creating', 'build...",java,train,"['▁', '@', 'Nonnull', '▁public', '▁static', '▁...",46.0,52.0


In [ ]:
# assert ((pd.concat([df_tst, df_trn, df_val]) == df).all()).all()

In [ ]:
# df_new_trn, df_bpe = train_test_split(df_trn, test_size = 0.1) # 10% selected to match the Maybe Deep Neural Networks are the Best Choice for Modeling Source Code paper

In [ ]:
# len(df_new_trn), len(df_bpe)

In [ ]:
# df_bpe.partition = ['bpe'] * len(df_bpe)

In [6]:
df_trn = df[df.partition == 'train'].copy()
df_bpe = df[df.partition == 'bpe'].copy()
df_val = df[df.partition == 'valid'].copy()
df_tst = df[df.partition == 'test'].copy()
df_bpe.head()

,path,url,code,code_tokens,docstring,docstring_tokens,language,partition,bpe32k,code_len,bpe32_len
repo,,,,,,,,,,,
lucee/Lucee,core/src/main/java/lucee/runtime/PageContextIm...,https://github.com/lucee/Lucee/blob/29b153fc4e...,<public> <void> _<do>Include<(>PageSource<[><]...,"['public', 'void', '_doInclude', '(', 'PageSou...",calling this method and in this case it should...,"['calling', 'this', 'method', 'and', 'in', 'th...",java,bpe,"['▁public', '▁void', '▁_', 'do', 'Include', '(...",339.0,423.0
exoplatform/jcr,exo.jcr.component.core/src/main/java/org/exopl...,https://github.com/exoplatform/jcr/blob/3e7f9e...,<protected> ResultSet findWorkspaceDataSize<(>...,"['protected', 'ResultSet', 'findWorkspaceDataS...",{@inheritDoc},['{'],java,bpe,"['▁protected', '▁ResultSet', '▁find', 'Workspa...",33.0,50.0
gliga/ekstazi,org.ekstazi.core/src/main/java/org/ekstazi/age...,https://github.com/gliga/ekstazi/blob/5bf4d39a...,<private> <static> File findToolsJar<(><)> <{>...,"['private', 'static', 'File', 'findToolsJar', ...",Finds tools.jar in JDK.\n\n@return File for to...,"['Finds', 'tools', '.', 'jar', 'in', 'JDK', '.']",java,bpe,"['▁private', '▁static', '▁File', '▁find', 'Too...",183.0,242.0
jroyalty/jglm,src/main/java/com/hackoeur/jglm/support/FastMa...,https://github.com/jroyalty/jglm/blob/9397c2fc...,<private> <static> <void> split<(><final> <dou...,"['private', 'static', 'void', 'split', '(', 'f...","Compute split[0], split[1] such that their sum...","['Compute', 'split', '[', '0', ']', 'split', '...",java,bpe,"['▁private', '▁static', '▁void', '▁split', '('...",101.0,106.0
brianwhu/xillium,core/src/main/java/org/xillium/core/ServicePla...,https://github.com/brianwhu/xillium/blob/e8dbf...,<public> <void> realize<(>ApplicationContext w...,"['public', 'void', 'realize', '(', 'Applicatio...",Reloads the platform with the given applicatio...,"['Reloads', 'the', 'platform', 'with', 'the', ...",java,bpe,"['▁public', '▁void', '▁real', 'ize', '(', 'App...",406.0,513.0


In [ ]:
df_new = pd.concat([df_new_trn, df_bpe, df_val, df_tst], ignore_index = True)
df_new.head()

In [33]:
len(df_new[df_new.partition == 'train']), len(df_new[df_new.partition == 'bpe']), len(df_new[df_new.partition == 'valid']), len(df_new[df_new.partition == 'test'])

(409005, 45446, 15328, 26909)

In [7]:
len(df_trn), len(df_bpe), len(df_val), len(df_tst)

(384868, 42719, 14605, 25011)

In [32]:
df_non_ascii = remove_non_ascii(df_new)
df_replaced = replace_special_tokens(df_non_ascii, java_special_tokens)
len(df_replaced[df_replaced.partition == 'train']), len(df_replaced[df_replaced.partition == 'bpe']), len(df_replaced[df_replaced.partition == 'valid']), len(df_replaced[df_replaced.partition == 'test'])

(405507, 45039, 15021, 26716)

In [35]:
df_replaced.to_csv(path/'searchnet/[codesearchnet-java-1597073966.81902].csv', sep = '~', index = False)

In [8]:
df_non_ascii = remove_non_ascii(df_bpe)
df_replaced = replace_special_tokens(df_non_ascii, java_special_tokens)
len(df_bpe)

42719

In [36]:
tokenizer = train_tokenizer(df_replaced[df_replaced.partition == 'bpe'])
tokenizer.save(str(path/'bpe/tokenizer-java.json'))
tokenizer.get_vocab_size()

10000

In [10]:
tokenizer = train_tokenizer(df_replaced, java_special_tokens)
# tokenizer.save(str(path/'bpe/tokenizer-java.json'))
tokenizer.get_vocab_size()

10000

In [4]:
from tokenizers import Tokenizer

new_tokenizer = Tokenizer.from_file(str(path/'bpe/tokenizer-java.json'))
new_tokenizer.encode(' <&&> <{><').tokens

['Ġ', '<&&>', 'Ġ', '<{>', 'Ġ<']

In [7]:
new_tokenizer.enable_padding()

In [ ]:
new_tokenizer.pad

In [8]:
new_tokenizer.padding

{'length': None,
 'pad_to_multiple_of': None,
 'pad_id': 0,
 'pad_token': '[PAD]',
 'pad_type_id': 0,
 'direction': 'right'}

In [13]:
new_tokenizer.token_to_id('<')

116

In [10]:
new_tokenizer.id_to_token(0)

'<abstract>'

In [38]:
new_tokenizer.get_vocab()

{'ged': 3155,
 'decess': 8629,
 'UTE': 6167,
 'external': 8455,
 'ingleton': 6260,
 'Ġmaven': 9464,
 '8': 112,
 'lock': 779,
 'Ġlimit': 3246,
 'xff': 3949,
 'vileged': 5194,
 'Ġbs': 6677,
 'Mon': 1876,
 'raph': 1615,
 'render': 5678,
 'ium': 6407,
 'Closure': 5252,
 'ssert': 1774,
 'IfNot': 8177,
 'riter': 879,
 'UN': 1230,
 'Ġchildren': 3980,
 'ĉĉĉ': 376,
 'TOKEN': 3881,
 'Ġgraph': 4846,
 'UUID': 2674,
 'getPosition': 9600,
 'Orientation': 7732,
 '<{><': 1203,
 'STOM': 9092,
 'Ġlogging': 9411,
 'Trans': 1051,
 'Ġe': 314,
 'CHAN': 4835,
 'bucket': 4818,
 'Epoch': 9319,
 'iv': 952,
 'AU': 4224,
 'GnuP': 9889,
 'RL': 5599,
 'logging': 9859,
 'ĠLong': 1996,
 'Ġ"_': 5341,
 'Ġdeserialize': 8581,
 'Ġeps': 7750,
 ">'\\": 1936,
 'getRight': 8545,
 'Ġpacket': 5826,
 'Ġfail': 4025,
 'Day': 2121,
 'ITI': 7580,
 "Ġ'<(>'": 6877,
 'ArrayIndex': 7890,
 'Focus': 8742,
 'Future': 1517,
 'ĠSimpleDateFormat': 7697,
 'Ġdelegate': 5248,
 'Ġrelationship': 9041,
 'release': 5844,
 "<(>'</>": 7036,
 'types': 

In [ ]:
# idx = 0
# df_beaut = beautify_code(df_trn, n = 10)
# df_replaced = replace_special_tokens(df_beaut, java_special_tokens)

# tokenizer = train_tokenizer(df_trn)
# encoded = tokenizer.encode(df_replaced.code.values[idx])
# print(df_replaced.code.values[idx])
# print('=' * 100)
# print(encoded.tokens)